In [ ]:
import yfinance as yf 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

TICKER = "SPY"

INTERVAL = "1h"
if INTERVAL == "1h":
    PERIOD = "730d"
else:
    PERIOD = "max"

STRATEGY = ["Volume"]

SHIFT = 5 

LOOKBACK = 10000

def get_data (ticker = TICKER, interval = INTERVAL, lookback = LOOKBACK):

    df = yf.download(ticker, interval = interval, period = PERIOD)
    df.columns = df.columns.get_level_values(0)
    df = df.reset_index(drop=True)
    
    return df.iloc[-lookback:, :]

# Add Features
# Add Target 
def add_target(shift = SHIFT):

    df["Target"] = (df["Close"].shift(-shift) - df["Close"])/(df["Close"]) * 100

    return df

# Fix Autocorrelation



# Generate Regression Output
# Validate Regression Output

def main():

    df = get_data()

    return df 

df = main()
df


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY,SPY
Date,,,,,,
